In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

import pandas as pd
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta

app = Flask(__name__)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/index2")
def index2():
    return render_template('index2.html')

@app.route("/index3")
def index3():
    return render_template('index3.html')

@app.route("/preface")
def preface():
    return render_template('preface.html')

@app.route("/festivals")
def festivals():
    sqlalchemyObj = db.engine.execute('select * from venues ORDER BY Name')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    # print(venues)
    dataset2 = []
    dict={}
    for i in venues:
        dict['Name'] = i[0]
        dict['Lat'] = i[1]
        dict['Lng'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        dict['Desc'] = i[5]
        # print(i)
        # print(dict)
        dataset2.append(dict.copy()) #markers.append(fld.copy())
    print(dataset2)
    return render_template('festivals.html', entries = dataset2)

@app.route("/dressing")
def dressing():
    return render_template('dressing.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')


################################################################################################################

# WEB ROUTES FOR FESTIVALS

################################################################################################################

@app.route("/1")
def festvial1():
    return render_template('1.html')


################################################################################################################

# WEB ROUTES FOR ARTISTS

################################################################################################################

@app.route("/mozaiyang")
def mozaiyang():
    return render_template('mozaiyang.html')




################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# APPLICATION TEST RUN AT PORT 9002

################################################################################################################

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "用户名"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("注册")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "用户名"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "密码"})
    submit = SubmitField("登录")

class Venues(db.Model):
    __tablename__ = 'venues'
    name = db.Column(db.String(25), primary_key=True)
    Lat = db.Column(db.Float)
    Lng = db.Column(db.Float)
    url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    desc = db.Column(db.Text)
    
@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index3"))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run('localhost', 9008)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9008/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Nov/2021 19:44:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:44:44] "GET /styles/bootstrap-4.1.2/popper.js HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:44:44] "GET /styles/bootstrap-4.1.2/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:44:44] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:44:45] "GET /festivals HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:44:45] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -


[{'Name': '【上海】2022鸡皮疙瘩音乐节', 'Lat': 31.270009, 'Lng': 121.505284, 'Url': 'https://map.baidu.com/poi/Modernskylab/@13526053.415,3645511.8,19z?uid=9e6159a8fe306960d518b7bb&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://img.dahepiao.com/uploads/190920/190920/141763-1Z92009205EI.jpg', 'Desc': '[上海]上海 MODERNSKY LAB'}, {'Name': '【南京】2021南京森林音乐节江北音乐季', 'Lat': 32.084071, 'Lng': 118.709111, 'Url': 'https://map.baidu.com/poi/%E6%89%AC%E5%AD%90%E6%B1%9F%E5%85%AC%E5%9B%AD/@13214779.694156624,3752743.761807229,11.45z?uid=4dfaf67d9c46cae61e3894aa&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://p4.itc.cn/q_70/images03/20210622/eccfcf23fb8340cca7840f25ba274db7.png', 'Desc': '[南京]江北新区扬子江公园'}, {'Name': '【广州】 2021国潮音乐嘉年华·广州站', 'Lat': 23.430868, 'Lng': 113.233549, 'Url': 'https://map.baidu.com/poi/%E5%B9%BF%E5%B

127.0.0.1 - - [28/Nov/2021 19:44:50] "GET /dressing HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:44:51] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:44:52] "GET /festivals HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:44:52] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -


[{'Name': '【上海】2022鸡皮疙瘩音乐节', 'Lat': 31.270009, 'Lng': 121.505284, 'Url': 'https://map.baidu.com/poi/Modernskylab/@13526053.415,3645511.8,19z?uid=9e6159a8fe306960d518b7bb&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://img.dahepiao.com/uploads/190920/190920/141763-1Z92009205EI.jpg', 'Desc': '[上海]上海 MODERNSKY LAB'}, {'Name': '【南京】2021南京森林音乐节江北音乐季', 'Lat': 32.084071, 'Lng': 118.709111, 'Url': 'https://map.baidu.com/poi/%E6%89%AC%E5%AD%90%E6%B1%9F%E5%85%AC%E5%9B%AD/@13214779.694156624,3752743.761807229,11.45z?uid=4dfaf67d9c46cae61e3894aa&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://p4.itc.cn/q_70/images03/20210622/eccfcf23fb8340cca7840f25ba274db7.png', 'Desc': '[南京]江北新区扬子江公园'}, {'Name': '【广州】 2021国潮音乐嘉年华·广州站', 'Lat': 23.430868, 'Lng': 113.233549, 'Url': 'https://map.baidu.com/poi/%E5%B9%BF%E5%B

...

127.0.0.1 - - [28/Nov/2021 19:47:58] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:01] "GET /festivals HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:01] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -


[{'Name': '【上海】2022鸡皮疙瘩音乐节', 'Lat': 31.270009, 'Lng': 121.505284, 'Url': 'https://map.baidu.com/poi/Modernskylab/@13526053.415,3645511.8,19z?uid=9e6159a8fe306960d518b7bb&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://img.dahepiao.com/uploads/190920/190920/141763-1Z92009205EI.jpg', 'Desc': '[上海]上海 MODERNSKY LAB'}, {'Name': '【南京】2021南京森林音乐节江北音乐季', 'Lat': 32.084071, 'Lng': 118.709111, 'Url': 'https://map.baidu.com/poi/%E6%89%AC%E5%AD%90%E6%B1%9F%E5%85%AC%E5%9B%AD/@13214779.694156624,3752743.761807229,11.45z?uid=4dfaf67d9c46cae61e3894aa&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://p4.itc.cn/q_70/images03/20210622/eccfcf23fb8340cca7840f25ba274db7.png', 'Desc': '[南京]江北新区扬子江公园'}, {'Name': '【广州】 2021国潮音乐嘉年华·广州站', 'Lat': 23.430868, 'Lng': 113.233549, 'Url': 'https://map.baidu.com/poi/%E5%B9%BF%E5%B

127.0.0.1 - - [28/Nov/2021 19:48:09] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:12] "GET /festivals HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:12] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -


[{'Name': '【上海】2022鸡皮疙瘩音乐节', 'Lat': 31.270009, 'Lng': 121.505284, 'Url': 'https://map.baidu.com/poi/Modernskylab/@13526053.415,3645511.8,19z?uid=9e6159a8fe306960d518b7bb&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://img.dahepiao.com/uploads/190920/190920/141763-1Z92009205EI.jpg', 'Desc': '[上海]上海 MODERNSKY LAB'}, {'Name': '【南京】2021南京森林音乐节江北音乐季', 'Lat': 32.084071, 'Lng': 118.709111, 'Url': 'https://map.baidu.com/poi/%E6%89%AC%E5%AD%90%E6%B1%9F%E5%85%AC%E5%9B%AD/@13214779.694156624,3752743.761807229,11.45z?uid=4dfaf67d9c46cae61e3894aa&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&newfrom=zhuzhan_webmap&querytype=detailConInfo&da_src=shareurl', 'Img_url': 'https://p4.itc.cn/q_70/images03/20210622/eccfcf23fb8340cca7840f25ba274db7.png', 'Desc': '[南京]江北新区扬子江公园'}, {'Name': '【广州】 2021国潮音乐嘉年华·广州站', 'Lat': 23.430868, 'Lng': 113.233549, 'Url': 'https://map.baidu.com/poi/%E5%B9%BF%E5%B

127.0.0.1 - - [28/Nov/2021 19:48:16] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:16] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:48:19] "GET /dressing HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:19] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 19:48:22] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 19:48:22] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
